## Download all reports from Manager

In [7]:
# Webbrowser automation
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Time
import time
# To work with files
import wget
import pyreadstat
import os
# Data wrangling
import pandas as pd 
import numpy as np
# Visualization
import seaborn as sns
import glob
from dotenv import load_dotenv

load_dotenv()

from utils.utils import sign_in_manager
from utils.utils import get_survey_list
from utils.utils import generate_excel_data
from utils.utils import download_raw_data

pd.set_option("display.max_columns", None)

In [2]:
# Remove every file in the data folder
files = os.listdir(os.environ["DATA_DIR"])
for file in files:
    if os.path.isfile(os.environ["DATA_DIR"] + file):
        print(f"Removing file {os.environ['DATA_DIR'] + file}")
        os.remove(os.environ["DATA_DIR"] + file)
try:
    # Connect to the remote webdriver
    options = webdriver.EdgeOptions()
    # options.headless = True
    driver = webdriver.Remote(os.environ["REMOTE_SERVER"], options = options)

    # Sign into Manager using the credentials
    sign_in_manager(driver, os.environ["SYNO_EMAIL"], os.environ["SYNO_PASSWORD"])
    survey_list = get_survey_list(driver, 4700)

    # Generate the raw data for all surveys
    for survey_id in survey_list["Id"]:
        print(f"Generating report for {survey_id}")
        generate_excel_data(driver, survey_id)

    # Get all download links for lastest files generated
    exported_file_links = []
    for survey_id in survey_list["Id"]:
        print(f"Getting the raw data file link for {survey_id}")
        exported_file_links.append(download_raw_data(driver, survey_id))

    # Take links and download all files
    for file in exported_file_links:
        downloaded_file = wget.download(file, "data/")
        print(f"File {downloaded_file} was downloaded")
    
except Exception as e:
    print(f"Exception in webdriver: {e}")
finally:
    print("Quitting webdriver session")
    driver.quit()

Generating report for 524065
Error token found!	 Resubmitting form again
Report for survey 524065 is being generated
Generating report for 368438
Report for survey 368438 is being generated
Generating report for 625173
Report for survey 625173 is being generated
Generating report for 431924
Report for survey 431924 is being generated
Generating report for 597650
Report for survey 597650 is being generated
Generating report for 165836
Report for survey 165836 is being generated
Generating report for 828252
Report for survey 828252 is being generated
Generating report for 952146
Report for survey 952146 is being generated
Getting the raw data file link for 524065
100% [........................................................................] 13254071 / 13254071File data//Raw Data 524065 2024-03-13.xlsx was downloaded
Getting the raw data file link for 368438
100% [..........................................................................] 6473902 / 6473902File data//Raw Data 368438 2024-

## Generated the merged report

In [11]:
data, meta = pyreadstat.read_sav("merged_data.sav")
data.drop(data.index, axis = "index", inplace = True)

for file in glob.glob(os.environ["DATA_DIR"] + '*.xlsx'):
    print(f"Processing file {file}")
    chunk = pd.read_excel(file)
    chunk.drop(0, axis = "index", inplace = True)
    new_column_names = ['guid','ip','panelist_id','identity_id','last_page','version','mode','locale','started','completed','loi','status','source','device','g','year_of_birth','postal_code']
    chunk.columns = new_column_names + list(chunk.columns[len(new_column_names):])

    # Replace the CONTROL2 code with a custom one depending on survey ID
    if file.__contains__("524065"): # Sweden
        chunk["CONTROL2"] = 1
    elif file.__contains__("828252"): # USA
        chunk["CONTROL2"] = 2
    elif file.__contains__("597650"): # Singapore
        chunk["CONTROL2"] = 3
    elif file.__contains__("165836"): # UK
        chunk["CONTROL2"] = 4
    elif file.__contains__("431924"): # Germany
        chunk["CONTROL2"] = 5
    elif file.__contains__("625173"): # India
        chunk["CONTROL2"] = 6
    elif file.__contains__("368438"): # Japan
        chunk["CONTROL2"] = 7
    elif file.__contains__("952146"): # China
        chunk["CONTROL2"] = 8
    data = pd.concat([data, chunk], axis = "index")
    print(f"File {file} merged")
print("All files were merged!")

# Create a new column for the week number
data["week"] = pd.to_datetime(data["completed"]).dt.isocalendar().week
# Uppercase postcodes in A3
data["A3"] = data["A3"].str.upper()

# Read the master list of codes and brands (all countries)
brands = pd.read_excel("4162 Codebook.xlsx", sheet_name = "BRANDS", dtype=str)
# Read the codebook for each brand (in English)
brands_codebook = pd.read_excel("4162 Codebook.xlsx", sheet_name="BRANDS_CODEBOOK", dtype=str)

# Read the list of statements per country
statements = pd.read_excel("4162 Codebook.xlsx", sheet_name="STATEMENTS", dtype=str)
# Read the codebook for the statements (in English)
statements_codebook = pd.read_excel("4162 Codebook.xlsx", sheet_name="STATEMENTS_CODEBOOK", dtype=str)

# Remove unicode characters
brands['Brand name'] = brands['Brand name'].apply(lambda x: str(x).replace('\xa0', ' '))
statements['Statement'] = statements['Statement'].apply(lambda x: str(x).replace('\xa0', ' '))

# Get the list of codes and their brands
brand_labels = {}
for item in brands.to_dict(orient = "split")["data"]:
    brand_labels[item[1]] = item[0]

# Get the list of codes and their labels in English
brand_codes = {}
for item in brands_codebook.to_dict("split")["data"]:
    brand_codes[item[0]] = item[1]

# Get the list of codes and their statements
statement_labels = {}
for item in statements.to_dict(orient = "split")["data"]:
    statement_labels[item[1]] = item[0]
    
# Get the list of codes and their labels in English
statement_codes = {}
for item in statements_codebook.to_dict(orient = "split")["data"]:
    statement_codes[item[0]] = item[1]

for i in range(0, 10):
    # Replace the statement label by their respective codes (in each language)
    data[f"FILTER3_{i}"].replace(statement_labels, inplace=True)
    meta.variable_value_labels[f"FILTER3_{i}"] = statement_codes

for i in range(0, 30):
    # Replace the brand labels by their respective codes
    data[f"FILTER_{i}"].replace(brand_labels, inplace=True)
    meta.variable_value_labels[f"FILTER_{i}"] = brand_codes

for i in range(0, 8):
    data[f"FILTER2_{i}"].replace(brand_labels, inplace=True)
    meta.variable_value_labels[f"FILTER2_{i}"] = brand_codes

data.to_excel("4700 Lynxeye merged data.xlsx", index = False)
# pyreadstat.write_sav(data, "4700 Lynxeye merged data.sav", variable_value_labels = meta.variable_value_labels)

Processing file data/Raw Data 368438 2024-03-13.xlsx


/home/kenneth/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


File data/Raw Data 368438 2024-03-13.xlsx merged
Processing file data/Raw Data 431924 2024-03-13.xlsx


/home/kenneth/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


File data/Raw Data 431924 2024-03-13.xlsx merged
Processing file data/Raw Data 524065 2024-03-13.xlsx


...

File data/Raw Data 165836 2024-03-13.xlsx merged
Processing file data/Raw Data 828252 2024-03-13.xlsx


/home/kenneth/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


File data/Raw Data 828252 2024-03-13.xlsx merged
Processing file data/Raw Data 625173 2024-03-13.xlsx


/home/kenneth/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


File data/Raw Data 625173 2024-03-13.xlsx merged
All files were merged!


## Reconciliate unmatched responses